In [ ]:
import torch
import matplotlib.pyplot as plt
from nerfstudio.utils.eval_utils import eval_setup
from pathlib import Path
import numpy as np
from nerfstudio.viewer.viewer import Viewer
from nerfstudio.configs.base_config import ViewerConfig
# config = Path("outputs/garfield_plushie/dig/2024-02-27_144117/config.yml")#pca 64
# config = Path("outputs/garfield_plushie/dig/2024-02-27_163355/config.yml")#pca 128
# config = Path("outputs/garfield_plushie/dig/2024-02-27_164651/config.yml")#v2 128 dim
# config = Path("outputs/garfield_plushie/dig/2024-02-27_170339/config.yml")##v2 128 dim, 7stride
# config = Path("outputs/garfield_plushie/dig/2024-02-28_180544/config.yml")#v2, 64 dim 7 stride
#all above s model
# config = Path("outputs/garfield_plushie/dig/2024-02-29_133302/config.yml")#v2, b model, 64 dim 7 stride
# config = Path("outputs/garfield_plushie/dig/2024-02-29_135125/config.yml")#v2, b model, 64 dim 14 stride
# config = Path("outputs/garfield_plushie/dig/2024-02-29_142826/config.yml")#v2, b model, 64 dim 14 stride, huber loss
# config = Path("outputs/louvre_statue/dig/2024-02-29_144220/config.yml")#v2, b model, 64 dim 14 stride, huber loss

config = Path("outputs/garfield_plushie/dig/2024-03-01_120451/config.yml")#with garfield
_,pipeline,_,_ = eval_setup(config)
dino_loader = pipeline.datamanager.dino_dataloader
if hasattr(pipeline,"garfield_pipeline"):
    v = Viewer(ViewerConfig(default_composite_depth=False),config.parent,pipeline.datamanager.get_datapath(),pipeline)

In [ ]:
from PIL import Image
from torchvision.transforms import ToTensor
import moviepy.editor as mpy
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import plotly
plotly.offline.init_notebook_mode()
video_path = Path("/home/justin/Downloads/garfield_move.mp4")
clip = mpy.VideoFileClip(str(video_path))
#print length of video in seconds
frame = clip.get_frame(6.5)
pil_image = ToTensor()(Image.fromarray(frame))

# image_path = Path("/home/justin/nerfstudio/data/colorful_mugs_colmap/images/frame_00010.jpg")
# image_path = Path("/home/justin/nerfstudio/data/louvre_statue/images/frame_00170.png")
# pil_image = ToTensor()(Image.open(image_path))
img_pca_feats = dino_loader.get_pca_feats(pil_image.unsqueeze(0)).cuda().squeeze()
cam,data = pipeline.datamanager.next_train(0)
outputs = pipeline.model.get_outputs_for_camera(cam)
outputs['dino'][outputs['dino_alpha']<.9] = 0
which_to_rgb_pca = torch.cat([outputs['dino'].view(-1,img_pca_feats.shape[-1]),img_pca_feats.view(-1,img_pca_feats.shape[-1])],dim=0)
# which_to_rgb_pca = outputs['dino'].view(-1,img_pca_feats.shape[-1])
_,_,rgb_pca = torch.pca_lowrank(which_to_rgb_pca.view(-1,which_to_rgb_pca.shape[-1]), q=3, niter=30)
from nerfstudio.utils.colormaps import apply_pca_colormap
_,axs = plt.subplots(1,4,figsize=(20,10))
axs[0].imshow(pil_image.permute(1,2,0).cpu().numpy())
axs[1].imshow(data["image"].cpu().numpy())
axs[2].imshow(apply_pca_colormap(img_pca_feats,rgb_pca).cpu().numpy())
axs[3].imshow(apply_pca_colormap(outputs["dino"],rgb_pca).cpu().numpy())
plt.show()
#same thing but with plotly
# fig = make_subplots(rows=1, cols=4)
# # Add images to each subplot
# fig.add_trace(go.Image(z=pil_image.permute(1, 2, 0).cpu().numpy()*255), row=1, col=1)
# fig.add_trace(go.Image(z=data["image"].cpu().numpy()*255), row=1, col=2)
# fig.add_trace(go.Image(z=apply_pca_colormap(img_pca_feats, rgb_pca).cpu().numpy()*255), row=1, col=3)
# fig.add_trace(go.Image(z=apply_pca_colormap(outputs["dino"], rgb_pca).cpu().numpy()*255), row=1, col=4)
# fig.update_layout(width=1500) 

In [ ]:
from matplotlib.patches import ConnectionPatch
pix_coords = [(10,30),(26,37)]
pix_matches = []
_,diff_axs = plt.subplots(2,len(pix_coords),figsize=(20,10))
for i,pc in enumerate(pix_coords):
    img_pca_feat = img_pca_feats[pc[0],pc[1],:]
    distance_img = (outputs['dino'] - img_pca_feat).pow(2).sum(dim=-1).sqrt()[...,None]
    img_distance = (img_pca_feats - img_pca_feat).pow(2).sum(dim=-1).sqrt()
    # print(img_distance.max().item(),img_distance.min().item())
    # print(distance_img.max().item(),distance_img.min().item())
    diff_axs[0,i].imshow(distance_img.cpu().numpy().squeeze(),vmin = 10,vmax=30)
    diff_axs[1,i].imshow(img_distance.cpu().numpy(),vmin = 10,vmax=30)
    argmin = torch.argmin(distance_img).cpu().numpy()
    argmin_coords = (argmin//distance_img.shape[1],argmin%distance_img.shape[1])
    pix_matches.append(argmin_coords)
fig,axs = plt.subplots(1,2,figsize=(20,10))
#visualize the input click on the original image
axs[0].imshow(apply_pca_colormap(img_pca_feats,rgb_pca).cpu().numpy())
axs[1].imshow(apply_pca_colormap(outputs["dino"],rgb_pca).cpu().numpy())
for i in range(len(pix_matches)):
    color = np.random.rand(3)
    transFigure = fig.transFigure.inverted()
    con = ConnectionPatch(xyA=(pix_coords[i][1],pix_coords[i][0]), xyB=(pix_matches[i][1],pix_matches[i][0]),
                           coordsA="data", coordsB="data",
                      axesA=axs[0], axesB=axs[1], color=color)
    fig.add_artist(con)
    axs[0].scatter(pix_coords[i][1],pix_coords[i][0],c=color,s=300)
    axs[1].scatter(pix_matches[i][1],pix_matches[i][0],c=color,s=300)
plt.show()